# 📘 Singular Value Decomposition (SVD)

## ❓ What is SVD ? 

Given any matrix $A \in \mathbb{R}^{m \times n}$, there exists a factorization:

$$
A = U \Sigma V^T
$$

Where:
- $U \in \mathbb{R}^{m \times m}$ is an orthogonal matrix (i.e., $U^T U = I$)
- $V \in \mathbb{R}^{n \times n}$ is an orthogonal matrix (i.e., $V^T V = I$)
- $\Sigma \in \mathbb{R}^{m \times n}$ is diagonal with non-negative entries $\sigma_1 \geq \sigma_2 \geq \dots \geq \sigma_r > 0$ on the diagonal and zeros elsewhere

## ✅ Key Idea

Every real matrix has a Singular Value Decomposition. Using this matrix decomposition, we can represent a matrix as a product of two orthogonal matrices and a diagonal matrix. Orthogonal and diagonal matrices are special, well-structured, and useful matrices with special properties. These special properties can be exploited in many situations which makes this decomposition particularly useful.

## ❓ Why Do We Factorize a Matrix This Way ?

SVD expresses a matrix as a product of:
- Two orthogonal transformations ($U$ and $V^T$)
- A diagonal scaling ($\Sigma$)

This allows us to:
- Analyze rank, null space, and range
- Perform dimensionality reduction and compression
- Solve linear systems robustly (including ill-posed ones)
- Compute the best low-rank approximation of a matrix

SVD generalizes diagonalization and works for **all** real matrices — square or not, full-rank or rank-deficient.


## 🧠 Mathematical Procedure

1. Compute $A^T A$ (symmetric and positive semi-definite).
2. Find eigenvectors $v_1, \dots, v_n$ and eigenvalues $\lambda_1 \geq \lambda_2 \geq \dots \geq \lambda_n \geq 0$ of $A^T A$.
3. Define singular values as $\sigma_i = \sqrt{\lambda_i}$.
4. For each $\sigma_i > 0$, compute $u_i = \frac{1}{\sigma_i} A v_i$.
5. The set $\{u_1, \dots, u_r\}$ is orthonormal. Extend to a full basis if needed.
6. Then:

$$
A = \sum_{i=1}^{r} \sigma_i u_i v_i^T
$$

Where each $\sigma_i u_i v_i^T$ is a rank-1 matrix, and $r = \text{rank}(A)$.


## 📌 Notes

- The decomposition **always exists**
- The singular values $\sigma_i$ are **unique**
- In reduced form:
  - $U \in \mathbb{R}^{m \times r}$
  - $\Sigma \in \mathbb{R}^{r \times r}$
  - $V^T \in \mathbb{R}^{r \times n}$


## ✅ Summary

- SVD exists for **every** real matrix
- It factors $A$ into orthogonal × diagonal × orthogonal
- It reveals rank, structure, and compressibility

# Implementation from scratch

In [4]:
import numpy as np

def SVD(A, threshold=1e-10):
    """
    This function compute the Singular Value Decomposition (SVD) of matrix A from scratch.
    Returns matrices U, sigma, and V_transpose such that A ≈ U @ Sigma @ V_transpose.

    Parameters:
    A : np.ndarray
        Input Matrix of shape (m, n)
    threshold : float
        Threshold below which singular values are treated as zero

    Returns:
    U : np.ndarray
        Left Singular Vectors (m x m)
    sigma : np.ndarray
        Diagonal matrix of singular values (m x n)
    V_transpose : np.ndarray
        Transpose of Right Singular vectors (n x n)
    """
    m, n = A.shape

    # Compute product of A^T and A
    B = A.T @ A

    # Find eigen values and eigen vectors of B
    eigen_values, V = np.linalg.eigh(B)

    # Sort eigen values and corresponding eigen vectors in descending order
    sorted_indices = np.argsort(eigen_values)[::-1]
    eigen_values = eigen_values[sorted_indices]
    V = V[:, sorted_indices]

    # Compute singular values
    singular_values = np.sqrt(np.clip(eigen_values, 0, None))

    # Compute U from A and V
    U = np.zeros((m, n))
    for i in range(n):
        # We are avoiding eigen vectors with extremely small or zero eigen values
        if singular_values[i] > threshold:
            U[:, i] = (A @ V[:, i]) / singular_values[i]

    # Normalize columns of U
    for i in range(n):
        norm = np.linalg.norm(U[:, i])
        if norm > threshold:
            U[:, i] /= norm

    # Compute sigma (m x n)
    sigma = np.zeros((m, n))
    np.fill_diagonal(sigma, singular_values)

    # V is already orthonormal ⇒ VT = V.T
    V_transpose = V.T

    return U, sigma, V_transpose

In [5]:
import pandas as pd

def SVD_test(num_tests: int = 10, shape: tuple = (3, 3), seed: int = 42) -> pd.DataFrame:
    """
    Generate and test SVD reconstruction accuracy on random matrices.

    Parameters:
    - num_tests: Number of random test cases
    - shape: Shape of the matrix (m, n)
    - seed: Random seed for reproducibility

    Returns:
    - DataFrame with absolute and relative reconstruction error per test
    """
    np.random.seed(seed)
    results = []
    
    for i in range(num_tests):
        A = np.random.randn(*shape)

        # Full SVD
        U, sigma, V_transpose = SVD(A)
        
        # Calculate A using SVD.
        k = min(A.shape)
        U_k = U[:, :k]                
        sigma_k = np.diag(sigma.diagonal()[:k])  
        V_k_T = V_transpose[:k, :]    
        A_reconstructed = U_k @ sigma_k @ V_k_T

        # Compute errors
        abs_error = np.linalg.norm(A - A_reconstructed, ord='fro')
        rel_error = abs_error / np.linalg.norm(A, ord='fro')

        results.append({
            "Test Case": i + 1,
            "Absolute Error": abs_error,
            "Relative Error (%)": rel_error * 100
        })

    return pd.DataFrame(results)

In [6]:
import pandas as pd

def SVD_test_example(example):
    """
    Takes a matrix as input and outputs the corresponding decomposition along with the original and calculated matrix 
    based on the decomposition to verify the correctness of SVD.

    Parameters :
     - example : Actual example matrix
    """
    # SVD
    U, sigma, V_transpose = SVD(example)
    
    # Calculate example using SVD.
    k = min(example.shape)
    U_k = U[:, :k]                
    sigma_k = np.diag(sigma.diagonal()[:k])  
    V_k_T = V_transpose[:k, :]    
    example_reconstructed = U_k @ sigma_k @ V_k_T

    # Print the results
    print("SVD Decomposition : ")
    print("U (Left Singular Vectors):\n", U)
    print("\nsigma (Diagonal Matrix of Singular Values):\n", sigma)
    print("\nV_transpose (Transpose of Right Singular Vectors):\n", V_transpose)

    # Compute errors
    print("Errors : ")
    abs_error = np.linalg.norm(example - example_reconstructed, ord='fro')
    rel_error = abs_error / np.linalg.norm(example, ord='fro')

    print("Absolute Error :", abs_error)
    print("Relative Error (%) :", rel_error * 100)

# Examples

## Example 1: 

$$
A = \begin{bmatrix} 3 & 1 \\ 1 & 3 \end{bmatrix}
$$

### Step 1: Compute $A^T A$

$$
A^T A = \begin{bmatrix} 10 & 6 \\ 6 & 10 \end{bmatrix}
$$

### Step 2: Eigenvalues of $A^T A$

Solve $\det(A^T A - \lambda I) = 0$:

$$
\lambda_1 = 16, \quad \lambda_2 = 4
$$

### Step 3: Singular values

$$
\sigma_1 = \sqrt{16} = 4, \quad \sigma_2 = \sqrt{4} = 2
$$

### Step 4: Right singular vectors $v_1, v_2$

$$
v_1 = \frac{1}{\sqrt{2}} \begin{bmatrix} 1 \\ 1 \end{bmatrix}, \quad
v_2 = \frac{1}{\sqrt{2}} \begin{bmatrix} -1 \\ 1 \end{bmatrix}
$$

### Step 5: Left singular vectors $u_1, u_2$

$$
u_1 = \frac{1}{4} A v_1 = \frac{1}{\sqrt{2}} \begin{bmatrix} 1 \\ 1 \end{bmatrix}, \quad
u_2 = \frac{1}{2} A v_2 = \frac{1}{\sqrt{2}} \begin{bmatrix} -1 \\ 1 \end{bmatrix}
$$

### Final SVD:

$$
U = \frac{1}{\sqrt{2}} \begin{bmatrix} 1 & -1 \\ 1 & 1 \end{bmatrix}, \quad
\Sigma = \begin{bmatrix} 4 & 0 \\ 0 & 2 \end{bmatrix}, \quad
V^T = \frac{1}{\sqrt{2}} \begin{bmatrix} 1 & 1 \\ -1 & 1 \end{bmatrix}
$$

**Note that the above answers are equivalent to the answers returned by the function**

In [9]:
example_1 = np.array([
    [3, 1],
    [1, 3]
], dtype=float)
SVD_test_example(example_1)

SVD Decomposition : 
U (Left Singular Vectors):
 [[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]

sigma (Diagonal Matrix of Singular Values):
 [[4. 0.]
 [0. 2.]]

V_transpose (Transpose of Right Singular Vectors):
 [[ 0.70710678  0.70710678]
 [-0.70710678  0.70710678]]
Errors : 
Absolute Error : 0.0
Relative Error (%) : 0.0


## Example 2

$$
A = \begin{bmatrix} 2 & 4 \\ 1 & 3 \\ 0 & 0 \end{bmatrix}
$$

### Step 1: Compute $A^T A$

$$
A^T A = \begin{bmatrix} 2 & 1 & 0 \\ 4 & 3 & 0 \end{bmatrix}
\begin{bmatrix} 2 & 4 \\ 1 & 3 \\ 0 & 0 \end{bmatrix}
= \begin{bmatrix} 5 & 10 \\ 10 & 25 \end{bmatrix}
$$

### Step 2: Eigenvalues of $A^T A$

Solve:

$$
A^T A = \begin{bmatrix} 2 & 1 & 0 \\ 4 & 3 & 0 \end{bmatrix}
\begin{bmatrix} 2 & 4 \\ 1 & 3 \\ 0 & 0 \end{bmatrix}
= \begin{bmatrix} 5 & 11 \\ 11 & 25 \end{bmatrix}
$$

Solve \( \det(A^T A - \lambda I) = 0 \):

$$
\lambda_1 \approx 29.864, \quad \lambda_2 \approx 0.134
$$

### Step 3: Singular values

$$
\sigma_1 = \sqrt{29.864} \approx 5.465, \quad \sigma_2 = \sqrt{0.134} \approx 0.366
$$

### Step 4: Right singular vectors $v_1, v_2$

Columns of \( V \) (or rows of \( V^T \)) are:

$$
v_1 \approx \begin{bmatrix} -0.4046 \\ -0.9145 \end{bmatrix}, \quad
v_2 \approx \begin{bmatrix} -0.9145 \\ 0.4046 \end{bmatrix}
$$

So:

$$
V^T \approx \begin{bmatrix}
-0.4046 & -0.9145 \\
-0.9145 & 0.4046
\end{bmatrix}
$$

### Step 5: Compute left singular vectors $u_i = \frac{1}{\sigma_i} A v_i$

Compute using \( u_i = \frac{1}{\sigma_i} A v_i \):

$$
u_1 \approx \frac{1}{5.465} A v_1 = \begin{bmatrix} -0.8174 \\ -0.5760 \\ 0 \end{bmatrix}, \quad
u_2 \approx \frac{1}{0.366} A v_2 = \begin{bmatrix} -0.5760 \\ 0.8174 \\ 0 \end{bmatrix}
$$

### Final SVD:

$$
U \approx \begin{bmatrix}
-0.8174 & -0.5760 \\
-0.5760 &  0.8174 \\
\phantom{-}0      &  0
\end{bmatrix}, \quad
\Sigma = \begin{bmatrix}
5.465 & 0 \\
0 & 0.366
\end{bmatrix}, \quad
V^T \approx \begin{bmatrix}
-0.4046 & -0.9145 \\
-0.9145 &  0.4046
\end{bmatrix}
$$
**Note that the above answers are equivalent to the answers returned by the function**

In [11]:
example_2 = np.array([
    [2, 4],
    [1, 3],
    [0, 0]
], dtype=float)
SVD_test_example(example_2)

SVD Decomposition : 
U (Left Singular Vectors):
 [[ 0.81741556 -0.57604844]
 [ 0.57604844  0.81741556]
 [ 0.          0.        ]]

sigma (Diagonal Matrix of Singular Values):
 [[5.4649857  0.        ]
 [0.         0.36596619]
 [0.         0.        ]]

V_transpose (Transpose of Right Singular Vectors):
 [[ 0.40455358  0.9145143 ]
 [-0.9145143   0.40455358]]
Errors : 
Absolute Error : 3.3306690738754696e-16
Relative Error (%) : 6.080941944488117e-15


## Example 3

$$
A = \begin{bmatrix} 4 & 8 \\ 2 & 4 \end{bmatrix}
$$

This is a rank-1 matrix (row 2 = 0.5 × row 1).

### Step 1: Compute $A^T A$

$$
A^T A = \begin{bmatrix} 20 & 40 \\ 40 & 80 \end{bmatrix}
$$

Eigenvalues: $\lambda_1 = 100, \quad \lambda_2 = 0$

### Step 2: Singular values

$$
\sigma_1 = 10, \quad \sigma_2 = 0
$$

### Step 3: Right singular vectors

$$
v_1 = \frac{1}{\sqrt{5}} \begin{bmatrix} 1 \\ 2 \end{bmatrix}, \quad
v_2 = \frac{1}{\sqrt{5}} \begin{bmatrix} -2 \\ 1 \end{bmatrix}
$$

### Step 4: Left singular vector

$$
u_1 = \frac{1}{\sigma_1} A v_1 = \frac{1}{\sqrt{5}} \begin{bmatrix} 1 \\ 0.5 \end{bmatrix}
$$

### Final SVD:

$$
U = \left[ \frac{1}{\sqrt{5}} \begin{bmatrix} 1 \\ 0.5 \end{bmatrix}, \ \text{(any orthogonal vector)} \right], \quad
\Sigma = \begin{bmatrix} 10 & 0 \\ 0 & 0 \end{bmatrix}, \quad
V^T = \begin{bmatrix}
\frac{1}{\sqrt{5}} & \frac{2}{\sqrt{5}} \\
\frac{-2}{\sqrt{5}} & \frac{1}{\sqrt{5}}
\end{bmatrix}
$$
**Note that the above answers are equivalent to the answers returned by the function**

In [13]:
example_3 = np.array([
    [4, 8],
    [2, 4]
], dtype=float)
SVD_test_example(example_3)

SVD Decomposition : 
U (Left Singular Vectors):
 [[0.89442719 0.        ]
 [0.4472136  0.        ]]

sigma (Diagonal Matrix of Singular Values):
 [[10.  0.]
 [ 0.  0.]]

V_transpose (Transpose of Right Singular Vectors):
 [[ 0.4472136   0.89442719]
 [-0.89442719  0.4472136 ]]
Errors : 
Absolute Error : 0.0
Relative Error (%) : 0.0


## Analysis Of Accuracy of SVD Function

Our function should ideally produce exact entries of the input matrix. But there are extremely small errors, due to some approximations that are made. These errors are too small that they can be usually ignored. Note that, SVD works and will produce exact results all the time. These small errors are due to approximations that we made. We have tested the function thoroughly and have added the results below to demonstrate the accuracy of our function.

## Case 1 (m > n) :

In [17]:
SVD_test(30, (10, 8))

,Test Case,Absolute Error,Relative Error (%)
0,1,6.395583e-15,7.452255e-14
1,2,5.748057e-15,7.047176e-14
2,3,6.479067e-15,7.024631e-14
3,4,4.496219e-15,5.239688e-14
4,5,7.425888e-15,8.922364e-14
5,6,6.024299e-15,6.293191e-14
6,7,6.930252e-15,8.495560e-14
7,8,7.958728e-15,8.475872e-14
8,9,7.223310e-15,7.832824e-14
9,10,5.173483e-15,6.030690e-14


## Case 2 (m < n) :

In [19]:
SVD_test(10, (8, 10))

,Test Case,Absolute Error,Relative Error (%)
0,1,8.196012e-15,9.550148e-14
1,2,7.093132e-15,8.696251e-14
2,3,7.814974e-15,8.473027e-14
3,4,6.043777e-15,7.043141e-14
4,5,8.358877e-15,1.004337e-13
5,6,8.786484e-15,9.178665e-14
6,7,9.733987e-15,1.193256e-13
7,8,8.595189e-15,9.153690e-14
8,9,8.265370e-15,8.962816e-14
9,10,5.847756e-15,6.816685e-14


## Case 3 (m = n) :

In [21]:
SVD_test(30, (8, 8))

,Test Case,Absolute Error,Relative Error (%)
0,1,5.350747e-15,7.346124e-14
1,2,1.855344e-14,2.367426e-13
2,3,6.355739e-15,8.612678e-14
3,4,6.095640e-15,7.086067e-14
4,5,1.256761e-14,1.680040e-13
5,6,1.093405e-14,1.550560e-13
6,7,7.635678e-15,8.850847e-14
7,8,5.869146e-15,7.183509e-14
8,9,4.453132e-15,5.713535e-14
9,10,1.142748e-14,1.428800e-13


# 📊 Applications in Quantitative Finance

Singular Value Decomposition (SVD) is a powerful linear algebra tool used widely in quantitative finance to analyze, de-noise, and simplify high-dimensional datasets.

## ✅ 1. Principal Component Analysis (PCA)

- **Use Case**: Dimensionality reduction of correlated asset returns.
- **Goal**: Capture most of the variance using a small number of uncorrelated components.

Steps:
1. Let $R \in \mathbb{R}^{n \times t}$ be the asset return matrix (n assets, t time steps).
2. Center the data (zero mean for each row).
3. Compute SVD: $R = U \Sigma V^T$
4. The columns of $V$ are principal components; singular values tell their importance.

📌 **Example**: Reduce 500 S&P 500 stocks to 3 main components (e.g., growth, value, volatility).

## ✅ 2. Factor Models and Risk Decomposition

- **Use Case**: Extract systematic risk factors.
- **Goal**: Express asset returns as a combination of a few orthogonal latent risk factors.

SVD helps identify these directions and quantify exposure.

📌 **Example**: Estimate factor exposures (like Fama–French) from historical returns using SVD.

## ✅ 3. Covariance Matrix Cleaning

- **Use Case**: Improve stability of portfolio optimization.
- **Goal**: Remove noise from empirical covariance estimates.

Steps:
1. Compute sample covariance matrix $\Sigma$
2. Perform SVD: $\Sigma = U \Sigma U^T$
3. Keep top $k$ singular values; zero out the rest
4. Reconstruct "cleaned" matrix

📌 **Example**: Avoid overfitting in mean-variance optimization by filtering noise.

## ✅ 4. Portfolio Compression / Approximation

- **Use Case**: Represent large portfolios using fewer synthetic instruments.
- **Goal**: Approximate exposure using fewer components.

SVD can express the portfolio weights matrix as a low-rank sum.

📌 **Example**: Replace 100-asset portfolio with 5 "principal" portfolios having similar exposure.

## ✅ 5. Latent Alpha Signal Discovery

- **Use Case**: Analyze alternative data (price-volume, sentiment, etc.)
- **Goal**: Extract dominant signals or latent factors.

SVD applied to a time-series × features matrix can reveal patterns not visible directly.

📌 **Example**: Decompose a matrix of trading signals to isolate the most predictive factors.

## 🧠 Summary Table

| Application                      | Purpose                              | Benefit                              |
|----------------------------------|--------------------------------------|--------------------------------------|
| PCA for Assets                   | Dimensionality reduction             | Better visualization & prediction    |
| Factor Models                    | Systematic risk extraction           | Explains returns more effectively    |
| Covariance Matrix Cleaning       | Noise reduction                      | More stable optimization             |
| Portfolio Compression            | Low-rank approximation               | Fewer instruments, same exposure     |
| Latent Alpha Signal Detection    | Feature decomposition                | Improved model interpretability      |

